In [36]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import statsmodels.api as sm
import patsy
from scipy.linalg import svd

In [37]:
def myeval():
    file_path = 'Proj2_Data/test_with_label.csv'
    test_with_label = pd.read_csv(file_path)
    num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'Proj2_Data/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'Proj2_Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)

        # Left join with the test data
        new_test = test_pred.merge(test, on=['Date', 'Store', 'Dept'], how='left')

        # Compute the Weighted Absolute Error
        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    return wae

In [38]:

def post_prediction_adjustment(test_pred, shift=1/7):
    # Define the critical weeks
    critical_weeks = ['2011-12-16', '2011-12-23', '2011-12-30', '2012-01-06', '2012-01-13']
    test_pred['Date'] = pd.to_datetime(test_pred['Date'])
    test_pred['Wk'] = test_pred['Date'].dt.isocalendar().week

    # average sales for weeks 49, 50, and 51
    avg_sales_49_51 = test_pred[test_pred['Date'].isin(['2011-12-02', '2011-12-09', '2011-12-16'])].groupby(['Store', 'Dept'])['Weekly_Pred'].mean().reset_index()

    # average sales for weeks 48 and 52
    avg_sales_48_52 = test_pred[test_pred['Date'].isin(['2011-11-25', '2011-12-30'])].groupby(['Store', 'Dept'])['Weekly_Pred'].mean().reset_index()

    merged_avg = pd.merge(avg_sales_49_51, avg_sales_48_52, on=['Store', 'Dept'], how='inner', suffixes=('_49_51', '_48_52'))

    # departments with sales bulge
    bulge_depts = merged_avg[merged_avg['Weekly_Pred_49_51'] > 1.1 * merged_avg['Weekly_Pred_48_52']]

    
    for date in critical_weeks:
        for _, row in bulge_depts.iterrows():
            store, dept = row['Store'], row['Dept']
            current_week_sales = test_pred[(test_pred['Date'] == date) & (test_pred['Store'] == store) & (test_pred['Dept'] == dept)]['Weekly_Pred']
            
            if not current_week_sales.empty:
                test_pred.loc[(test_pred['Date'] == date) & (test_pred['Store'] == store) & (test_pred['Dept'] == dept), 'Weekly_Pred'] *= (1 - shift)
                
                next_week = (pd.to_datetime(date) + pd.Timedelta(weeks=1)).strftime('%Y-%m-%d')
                test_pred.loc[(test_pred['Date'] == next_week) & (test_pred['Store'] == store) & (test_pred['Dept'] == dept), 'Weekly_Pred'] += current_week_sales.values[0] * shift

    return test_pred

def preprocess(data):
    data = data.copy()
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 
#    data['IsHoliday'] = data['IsHoliday'].apply(int)
    return data
    
for fold_num in range(1,11):
    # Reading train data
    file_path = f'Proj2_Data/fold_{fold_num}/train.csv'
    train = pd.read_csv(file_path)
    
    # Reading test data
    file_path = f'Proj2_Data/fold_{fold_num}/test.csv'
    test = pd.read_csv(file_path)
    
    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()
    
    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])
    
    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
    train_split = preprocess(train_split)
    y, X = patsy.dmatrices('Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr  + Wk', 
                           data = train_split, 
                           return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))
    
    
    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk', 
                           data = test_split, 
                           return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))
    
    keys = list(train_split)
    
    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]
     
        Y = X_train['Weekly_Sales']
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)
        
        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
     
        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values
    
            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-10:
                    cols_to_drop.append(col_name)
                
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
    
        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)
        
        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)
        
        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)
        
    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    test_pred = post_prediction_adjustment(test_pred, shift=1/7)
    print(test_pred.shape)
    file_path = f'Proj2_Data/fold_{fold_num}/mypred.csv'
    test_pred.to_csv(file_path, index=False)

(26498, 5)
(23532, 5)
(26355, 5)
(26551, 5)
(26838, 5)
(23780, 5)
(26723, 5)
(26570, 5)
(26588, 5)
(23720, 5)


In [39]:
print(test_pred)

       Store  Dept       Date   Weekly_Pred  Wk
0        1.0   1.0 2012-09-07  18043.059966  36
1        1.0   1.0 2012-09-14  19017.204966  37
2        1.0   1.0 2012-09-21  18063.259966  38
3        1.0   1.0 2012-09-28  19288.284966  39
4        1.0   1.0 2012-10-05  22165.154966  40
...      ...   ...        ...           ...  ..
23715   45.0  98.0 2012-09-28    906.834050  39
23716   45.0  98.0 2012-10-05   1020.944050  40
23717   45.0  98.0 2012-10-12   1070.664050  41
23718   45.0  98.0 2012-10-19   1169.634050  42
23719   45.0  98.0 2012-10-26   1234.404050  43

[23720 rows x 5 columns]


In [42]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

	2049.347
	1467.113
	1446.882
	1595.628
	2154.461
	1675.221
	1720.828
	1427.286
	1443.787
	1444.677
1642.523
